<a href="https://colab.research.google.com/github/zackives/upenn-cis5450-hw/blob/main/14_Module_3_Notebook_II_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finding Clusters



## Autograder setup

In [ ]:
#PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO
#TO ASSIGN POINTS TO YOU IN OUR BACKEND
STUDENT_ID = 99999999 # YOUR PENN-ID GOES HERE AS AN INTEGER##PLEASE ENSURE YOUR PENN-ID IS ENTERED CORRECTLY. IF NOT, THE AUTOGRADER WON'T KNOW WHO

In [ ]:
%%writefile notebook-config.yaml

grader_api_url: 'https://23whrwph9h.execute-api.us-east-1.amazonaws.com/default/Grader23'
grader_api_key: 'flfkE736fA6Z8GxMDJe2q8Kfk8UDqjsG3GVqOFOa'

In [ ]:
%set_env HW_ID=cis5450_25f_HW9

In [ ]:
!pip3 install penngrader-client

In [ ]:
import os
from penngrader.grader import *

grader = PennGrader('notebook-config.yaml', os.environ['HW_ID'], STUDENT_ID, STUDENT_ID)

## Looking at Glass... ... Again

Recall the glass dataset from the PCA notebook.

In [ ]:
# Glass data from https://archive.ics.uci.edu/ml/machine-learning-databases/glass/

In [ ]:
!wget https://archive.ics.uci.edu/static/public/42/glass+identification.zip

In [ ]:
!unzip glass+identification.zip

In [ ]:
# Load into a dataframe, with the header in row 0
import pandas as pd

glass_df = pd.read_csv('glass.data',header=None,names=['ID','RefractiveIndex','Na','Mg','Al','Si','K','Ca','Ba','Fe','Label'])

glass_df

In [ ]:
glass_df.set_index('ID')
glass_types_df = glass_df[['Label']]

# We don't really need these
glass_df = glass_df.drop(columns=['ID', 'Label'])

display(glass_df)
display(glass_types_df)

## Unsupervised Machine Learning!

We typically set up machine learning problems as follows.

1. Convert from categorical and other values into numeric values
2. Convert from dataframes to arrays
3. Separate out any classes / labels (like glass type)

We will call the *input data*  $X$ and the *labels* $y$.

In [ ]:
# Set up the problem

X = glass_df.to_numpy()
y = glass_types_df.to_numpy()

## Running PCA

We'll use, for the first time, a standard sckikit-learn 'flow': create a model, `fit` it, and `transform` the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

# Standardizing the features based on unit variance
X = StandardScaler().fit_transform(X)

print (X.shape)
print(X)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=6)
X_embedded = pca.fit_transform(X)[:,1:3]

## Clustering Algorithm

### The Algorithm, in Full Detail

Let's start with the basic algorithm.  We'll split it into three components:
1. The clustering coefficient or `error` function -- how far away are points?
2. The function to get the most appropriate cluster for a point, `get_nearest`.
3. The main K-Means algorithm.

For this version we will initialize with randomly chosen points in the dataset, then iteratively recompute until we reach convergence (which is detected when every point remains in its current cluster).

In [ ]:
from random import randint
from scipy.spatial import distance
import numpy as np
import matplotlib.pyplot as plt

def error(x,c):
  """
  Error (distance, clustering coefficient) between a point x
  and the centroid c.  We are using Euclidean distance, which
  can also be thought of as the sum of the squared error.
  """
  err = np.linalg.norm(x - c)

  return err

def get_nearest(c_list, x):
  nearest = -1
  nearest_error = np.inf
  for i, v in enumerate(c_list):
    if error(x, v) < nearest_error:
      nearest_error = error(x, v)
      nearest = i
  return nearest

def kmeans(X, k, show):
  """
  Simple k-means algorithm
  """
  # Initialize the centroids to random points
  # in the data
  centroids = np.zeros((k,2))
  cluster_assignments = [0 for i in range(len(X))]
  for i in range(0, k):
    centroids[i] = X[randint(0, X.shape[0])]

  if show:
    print('Initialized centroids to: ')
    print(centroids)

  iteration = 1
  changed = True
  while changed:
    if show:
      print ('Iteration %d'%iteration)
      iteration = iteration + 1

    # Nothing happened in this iteration, by default
    changed = False
    # Assign points to clusters
    for i,x in enumerate(X):
      nearest = get_nearest(centroids, x)
      # We changed a cluster mapping!
      if nearest != cluster_assignments[i]:
        changed = True
      cluster_assignments[i] = nearest

    if changed:
      # Recompute clusters
      for i in range(len(centroids)):
        points = [j for j,v in enumerate(cluster_assignments) if v == i]
        if show:
          print ('Cluster %d'%i)
        X_subset = np.array([[X[i,0],X[i,1]] for i in points])

        if len(X_subset):
          centroids[i][0] = np.sum(X_subset[:, 0]) / len(points)
          centroids[i][1] = np.sum(X_subset[:, 1]) / len(points)
          print (centroids[i])
    elif show:
      print('Converged!')

  return (centroids, np.array(cluster_assignments))


k = 2
plt.scatter(X_embedded[:,0], X_embedded[:,1], c='red', marker='o', s=50)
plt.grid()
plt.show()
centroids, assignments = kmeans(X_embedded, k, True)

assignments

In [ ]:
# Plotting along with the Centroids
clust0 = np.array([x for i, x in enumerate(X_embedded) if assignments[i] == 0])
clust1 = np.array([x for i, x in enumerate(X_embedded) if assignments[i] == 1])

plt.scatter(clust0[:,0], clust0[:,1], c='red', marker='o', s=50)
plt.scatter(clust1[:,0], clust1[:,1], c='black', marker='o', s=50)
plt.scatter(centroids[:,0], centroids[:,1], marker='*', s=50, c='g')

In [ ]:
from sklearn.cluster import KMeans

km = KMeans(2, init='random', n_init=1, max_iter=300, random_state=0)
km.fit(X_embedded)

In [ ]:
km.cluster_centers_

In [ ]:
km.labels_

In [ ]:
# Plotting along with the Centroids
clust0 = np.array([x for i, x in enumerate(X_embedded) if km.labels_[i] == 0])
clust1 = np.array([x for i, x in enumerate(X_embedded) if km.labels_[i] == 1])

plt.scatter(clust0[:,0], clust0[:,1], c='red', marker='o', s=50)
plt.scatter(clust1[:,0], clust1[:,1], c='black', marker='o', s=50)
plt.scatter(centroids[:,0], centroids[:,1], marker='*', s=50, c='g')

### k-Means in SQL

For this one we'll use the college dataset from Kaggle.  Here we have both private and public universities, as well as many statistics.

The hope is that we can find 2 natural clusters.  Let's see!

In [ ]:
%set_env SPARK_VERSION=3.5.6

In [ ]:
## Let's install Apache Spark on Colab

!wget -nc https://downloads.apache.org/spark/spark-$SPARK_VERSION/spark-$SPARK_VERSION-bin-hadoop3.tgz
!tar xf spark-$SPARK_VERSION-bin-hadoop3.tgz
!pip install findspark

import os

os.environ["SPARK_HOME"] = "/content/spark-" + os.environ['SPARK_VERSION'] + "-bin-hadoop3"

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SQLContext

spark = SparkSession.builder.appName('Clustering').getOrCreate()

In [ ]:
import pandas as pd

# Sample data
km_data = pd.DataFrame(pd.read_csv('https://www.cis.upenn.edu/~zives/college-data.csv'))
km_data['private'] = km_data['private'] == 'Yes'

km_data.info()


In [ ]:
from sklearn.preprocessing import StandardScaler
from pyspark.sql.types import *

km_schema = StructType([StructField("index", IntegerType(), True),
                        StructField("private", FloatType(), True),
                        StructField("apps", FloatType(), True),
                        StructField("accept", FloatType(), True),
                        StructField("enroll", FloatType(), True),
                        StructField("top10perc", FloatType(), True),
                        StructField("top25perc", FloatType(), True),
                        StructField("f_undergrad", FloatType(), True),
                        StructField("p_undergrad", FloatType(), True),
                        StructField("outstate", FloatType(), True),
                        StructField("room_board", FloatType(), True),
                        StructField("books", FloatType(), True),
                        StructField("personal", FloatType(), True),
                        StructField("phd", FloatType(), True),
                        StructField("terminal", FloatType(), True),
                        StructField("s_f_ratio", FloatType(), True),
                        StructField("perc_alumni", FloatType(), True),
                        StructField("expend", FloatType(), True),
                        StructField("grad_rate", FloatType(), True)
                        ])

# Standardizing the features based on unit variance
km_data_2 = StandardScaler().fit_transform(km_data)
km_data_2_df = pd.DataFrame(km_data_2)
km_data_2_df.reset_index(inplace=True)
#km_data_2_df.info()
#print(km_data_2_df)
km_data_2_df.info()

km_data_sdf = spark.createDataFrame(km_data_2_df, km_schema)
km_data_sdf.show()

#km_data.info()


In [ ]:
km_data_sdf.schema

In [ ]:
k = 2
km_data_sdf.createOrReplaceTempView('km_data')

def initialize():
  ######################
  ## Initialize clusters.  It's better to do this randomly, but we'll use the
  ## first k=2 rows
  km_cluster_centroids_sdf = spark.sql("""
            SELECT index as cluster_id,apps,accept,enroll,top10perc,top25perc,f_undergrad,p_undergrad,outstate,room_board,books,personal,phd,terminal,s_f_ratio,perc_alumni,expend,grad_rate
            FROM km_data TABLESAMPLE (""" + str(k) + ' ROWS)')

  km_cluster_centroids_sdf.show()
  km_cluster_centroids_sdf.createOrReplaceTempView('km_cluster_centroids')
  return km_cluster_centroids_sdf

In [ ]:
def assign_clusters():
      # Distance between each point & each centroid
    km_data_to_cluster_dist_sdf =\
      spark.sql("""
                SELECT c.cluster_id,k.index,(pow(c.apps - k.apps, 2)+
                                              pow(c.accept - k.accept, 2)+
                                              pow(c.enroll - k.enroll, 2)+
                                              pow(c.top10perc - k.top10perc, 2)+
                                              pow(c.top25perc - k.top25perc, 2)+
                                              pow(c.f_undergrad - k.f_undergrad, 2)+
                                              pow(c.p_undergrad - k.p_undergrad, 2)+
                                              pow(c.outstate - k.outstate, 2)+
                                              pow(c.room_board - k.room_board, 2)+
                                              pow(c.personal - k.personal, 2)+
                                              pow(c.phd - k.phd, 2)+
                                              pow(c.terminal - k.terminal, 2)+
                                              pow(c.s_f_ratio - k.s_f_ratio, 2)+
                                              pow(c.perc_alumni - k.perc_alumni, 2)+
                                              pow(c.expend - k.expend, 2)+
                                              pow(c.grad_rate - k.grad_rate, 2)
                ) as dist
                FROM km_cluster_centroids c CROSS JOIN km_data k
                """)
    km_data_to_cluster_dist_sdf.createOrReplaceTempView('km_data_to_cluster_dist')

    # SparkSQL is very limited in subqueries.  We want for each index,
    # the cluster_id which minimizes the cluster distance.  We do two steps for
    # each data item: (1) find the shortest distance to ANY cluster ID,
    # (2) find the cluster ID whose distance equals the shortest distance.
    km_data_to_cluster_best_dist_sdf = spark.sql('''
              SELECT index,min(dist) as dist
              FROM km_data_to_cluster_dist
              GROUP BY index''')
    km_data_to_cluster_best_dist_sdf.createOrReplaceTempView('km_data_to_cluster_best_dist')

    km_data_to_cluster_sdf = spark.sql('''
              SELECT kd.index, kd.cluster_id
              FROM km_data_to_cluster_dist kd JOIN km_data_to_cluster_best_dist kb
                ON kd.index = kb.index
              WHERE kd.dist = kb.dist''')
    km_data_to_cluster_sdf.createOrReplaceTempView('km_data_to_cluster')
    km_data_to_cluster_sdf.show()
    return km_data_to_cluster_sdf


In [ ]:
def compute_centroids():
  km_clusters_sdf = spark.sql('''
            SELECT cluster_id,AVG(apps) AS apps,AVG(accept) AS accept,
               AVG(enroll) AS enroll,AVG(top10perc) AS top10perc,
               AVG(top25perc) AS top25perc,AVG(f_undergrad) AS f_undergrad,
               AVG(p_undergrad) AS p_undergrad,AVG(outstate) AS outstate,
               AVG(room_board) AS room_board,AVG(books) AS books,
               AVG(personal) AS personal,AVG(phd) AS phd,
               AVG(terminal) AS terminal,AVG(s_f_ratio) AS s_f_ratio,
               AVG(perc_alumni) AS perc_alumni,AVG(expend) AS expend,
               AVG(grad_rate) AS grad_rate
            FROM km_data_to_cluster kc JOIN km_data kd ON kc.index = kd.index
            GROUP BY kc.cluster_id
            ''')
  km_clusters_sdf.createOrReplaceTempView('km_cluster_centroids')
  km_clusters_sdf.show()
  return km_clusters_sdf

In [ ]:
max_iter = 1
print('Initial data:')
spark.sql('select * from km_data').show(5)
print('Initial cluster centroids:')
km_cluster_centroids_sdf = initialize()
km_cluster_centroids_sdf.show()
for i in range(max_iter):
  km_data_to_cluster_sdf = assign_clusters()
  # print ('Data-cluster assignments')
  # spark.sql('select * from km_data_to_cluster order by index').show(5)
  km_cluster_centroids_sdf = compute_centroids()
  # print ('New cluster centroids')
  # spark.sql('select * from km_cluster_centroids').show()

km_cluster_centroids_sdf.explain()


## Clustering in MLlib

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler


# Convert from dataframe features to a "features" column that is a vector
vecAssembler = VectorAssembler(inputCols=['apps','accept','enroll',\
                                          'top10perc','top25perc','f_undergrad',\
                                          'p_undergrad','outstate','room_board',\
                                          'books','personal','phd','terminal',\
                                          's_f_ratio','perc_alumni','expend',\
                                          'grad_rate'], outputCol="features")
vecAssembler = VectorAssembler(inputCols=['apps','accept','enroll',\
                                          'top10perc','top25perc','f_undergrad',\
                                          'p_undergrad','outstate','room_board',\
                                          'books','personal','phd','terminal',\
                                          's_f_ratio','perc_alumni','expend',\
                                          'grad_rate'], outputCol="features")
df_kmeans = vecAssembler.transform(km_data_sdf).select('index', 'features')
df_kmeans.show()

# Trains a k-means model.
kmeans = KMeans().setK(2).setSeed(5)

model = kmeans.fit(df_kmeans)

predictions = model.transform(df_kmeans)
predictions.show()

## Choosing the Right *k* for Clustering

To pick the right value of *k* for our data, we will search the space of possible values -- looking at the smallest one that (roughly) minimizes the sum squared error (Euclidean distance), aka the distortion.

In [ ]:
X_embedded

In [ ]:
distortions = []

# Convert X_embedded to Spark DF
num_features = X_embedded.shape[1]
feature_cols = [f"feature_{i}" for i in range(num_features)]

X_pdf = pd.DataFrame(X_embedded, columns=feature_cols)
X_sdf = spark.createDataFrame(X_pdf)
assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

final_sdf = assembler.transform(X_sdf)

max_k = 10
for i in range(2,max_k+1):
  km = KMeans(k=i,
              maxIter=300,
              seed=0)
  model = km.fit(final_sdf)
  # The distortion is called inertia in SciKit and trainingCost in MLLib
  distortions.append(model.summary.trainingCost)

plt.plot(range(2,max_k+1), distortions, marker='o')
plt.xlabel('Cluster count (k)')
plt.ylabel('Distortion')
plt.show()

## Exercise

Let's switch back to SciKit rather than MLlib.

Go back to the original glass data, re-run PCA *with no losses*, and find the optimal number of clusters.  Use the k-means++ algorithm (see [here](https://scikit-learn.org/1.5/modules/generated/sklearn.cluster.KMeans.html) for the parameters).  Search for up to 15 clusters.

You should select the "elbow" as the point where the slope of the line changes from a steeper to a shallower point.

In [ ]:
from sklearn.decomposition import PCA

X_embedded = #TODO

In [ ]:
distortions = []

max_k = 15
# TODO

plt.plot(range(1,max_k+1), distortions, marker='o')
plt.xlabel('Cluster count (k)')
plt.ylabel('Distortion')
plt.show()

In [ ]:
# How many dimensions should we use, to get 95% explained variance ratio?
clusters = # TODO for the number of clusters
clusters

In [ ]:
grader.grade('kmeans', clusters)